In [30]:
# Install dependencies
%pip install pennylane
%pip install qiskit
%pip install numpy pandas matplotlib scipy
%pip install folium
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Project - Quantum Approach to Biodiversity Mapping & Predictions

Figma [here](https://www.figma.com/board/YJkl666NgYY9lzeGnKZ1lw/Quantum-Hackathon-2024?node-id=0-1&node-type=canvas&t=7ul1ZMUwhKcWZdU6-0)

Goals:  
1.  
2.  

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import pennylane as qml
import qiskit as qk
import zipfile
import io
import os
import folium
import ipywidgets as widgets

from folium.plugins import TimestampedGeoJson
from IPython.display import display
from ipywidgets import interact, IntSlider

# Data

[BioTIME database](https://zenodo.org/records/5026943#.Y9ZAKdJBwUE)

Data Citation:
Dornelas M, Antão LH, Moyes F, Bates, AE, Magurran, AE, et al. BioTIME: A database of biodiversity time series for the Anthropocene. Global Ecol Biogeogr. 2018; 27:760 - 786. https://doi.org/10.1111/geb.12729

## Load Data

metadata and biodiversity data
see [here](https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a)
effectively need to upload the data to gdrive as collab storage is transient

In [4]:
metadata_df  = pd.read_csv("BioTIMEMetadata_24_06_2021.csv", encoding='latin1')
metadata_df.head()

,STUDY_ID,REALM,CLIMATE,GENERAL_TREAT,TREATMENT,TREAT_COMMENTS,TREAT_DATE,HABITAT,PROTECTED_AREA,BIOME_MAP,...,WEB_LINK,DATA_SOURCE,METHODS,SUMMARY_METHODS,LINK_ID,COMMENTS,DATE_STUDY_ADDED,ABUNDANCE_TYPE,BIOMASS_TYPE,SAMPLE_DESC_NAME
0,10,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Woodland,False,Temperate broadleaf and mixed forests,...,http://esapubs.org/archive/ecol/E082/011/defau...,Ecology,Itasca State Park. Minnesota. is a 13 000-ha f...,Plots,NaN,Dates added,Oct-12,Count,NaN,lat_long_treefallid_controlvs.treatment_basevs...
1,18,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Sagebrush steppe,False,Deserts and xeric shrublands,...,http://esapubs.org/archive/ecol/E091/243/defau...,Ecology,Site descriptionThe U.S. Sheep Experiment Stat...,Plots,NaN,Dates added Years (1923. 1926. 1927. 1929) del...,Oct-12,Count,NaN,lat_long_quadrat_year
2,33,Marine,Temperate,NaN,NaN,NaN,NaN,Seaweed beds,False,Temperate shelf and seas ecoregions,...,http://plankt.oxfordjournals.org/content/32/5.toc,Oxford Journals,Sampling and enumeration of phytoplanktonWeekl...,Stations,NaN,Inaccurate decimal latitude/longitude conversi...,Oct-12,Count,NaN,lat_long_location_method_date
3,39,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Deciduous forest,False,Temperate broadleaf and mixed forests,...,http://www.esajournals.org/toc/emon/56/3,Ecology,This study was conducted in the Hubbard Brook ...,Plots,NaN,Removed records prior to 1986 as per provider ...,Oct-12,Density,NaN,lat_long_timeTransect_year
4,41,Terrestrial,Temperate,NaN,NaN,NaN,NaN,Woodland,False,Temperate broadleaf and mixed forests,...,http://www.esajournals.org/toc/ecol/41/4,Ecology,Time and space and the variation of species - ...,Counts,NaN,Dates added,Oct-12,Count,NaN,lat_long_census_year


In [5]:
# with zipfile.ZipFile("BioTIMEQuery_24_06_2021.zip") as z:
#     print(z.namelist())
#     with z.open("BioTIMEQuery_24_06_2021.csv") as f:
#         data_df = pd.read_csv(f, encoding='latin1')
data_df = pd.read_csv("BioTIMEQuery_24_06_2021.csv", encoding='latin1')
data_df.head()

/tmp/ipykernel_3256634/4200733232.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv("BioTIMEQuery_24_06_2021.csv", encoding='latin1')


,Unnamed: 0,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
0,1,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,22,47.4,-95.12,1.0,0.0,Acer,rubrum,Acer rubrum
1,2,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,23,47.4,-95.12,3.0,0.0,Acer,saccharum,Acer saccharum
2,3,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,24,47.4,-95.12,1.0,0.0,Acer,spicatum,Acer spicatum
3,4,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,607,47.4,-95.12,12.0,0.0,Corylus,cornuta,Corylus cornuta
4,5,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Small,12,1911,47.4,-95.12,1.0,0.0,Populus,pinnata,Populus pinnata


In [6]:
#data_df.shape
data_df.tail()

,Unnamed: 0,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
8552244,26178100,548,NaN,NaN,2007,49.1014548954342_13.3200349605548_T3_56_2007,T3_56,49340,49.10146,13.32004,3.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552245,26179100,548,NaN,NaN,2009,49.1014548954342_13.3200349605548_T3_56_2009,T3_56,49340,49.10146,13.32004,4.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552246,26180100,548,NaN,NaN,2012,49.1014548954342_13.3200349605548_T3_56_2012,T3_56,49340,49.10146,13.32004,3.0,NaN,Vaccinium,vitis.idaea,Vaccinium vitis.idaea
8552247,26181100,548,NaN,NaN,2007,49.097317976565_13.3173542074378_T3_51_2007,T3_51,40355,49.09732,13.31735,10.0,NaN,Veronica,chamaedrys,Veronica chamaedrys
8552248,26182100,548,NaN,NaN,2011,49.1014548954342_13.3200349605548_T3_56_2011,T3_56,40355,49.10146,13.32004,4.0,NaN,Veronica,chamaedrys,Veronica chamaedrys


# Visualisation

In [7]:
map_center = [data_df['LATITUDE'].mean(), data_df['LONGITUDE'].mean()]
m = folium.Map(location=map_center, zoom_start=2)
m

In [10]:
# Strip any leading/trailing spaces from column names
data_df.columns = data_df.columns.str.strip()

# Function to update the map based on the selected year
def update_map(year):
    # Filter data for the selected year
    filtered_data = data_df[data_df['YEAR'] == year]
    
    # Create a map centered around the mean latitude and longitude
    map_center = [filtered_data['LATITUDE'].mean(), filtered_data['LONGITUDE'].mean()]
    m = folium.Map(location=map_center, zoom_start=2)
    
    # Add markers to the map
    for _, row in filtered_data.iterrows():
        folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            popup=row['SPECIES']
        ).add_to(m)
    
    # Display the map
    display(m)

# Create a year slider
year_slider = IntSlider(min=data_df['YEAR'].min(), max=data_df['YEAR'].max(), step=1, description='Year')

# Use the interact function to update the map based on the selected year
interact(update_map, year=year_slider)


interactive(children=(IntSlider(value=1874, description='Year', max=2018, min=1874), Output()), _dom_classes=(…

<function __main__.update_map(year)>

In [ ]:
for i, row in data_df.iterrows():
    folium.CircleMarker(
        location=(row['LATITUDE'], row['LONGITUDE']),
        radius=row['sum.allrawdata.ABUNDANCE'] ** 0.5,
        popup=(f"Biomass: {row['sum.allrawdata.BIOMASS']}<br>Abundance: {row['sum.allrawdata.ABUNDANCE']}"),
        color='green',
        fill=True,
        fill_color='teal',
        fill_opacity=0.6
    ).add_to(m)
m

: 